<a href="https://colab.research.google.com/github/priyankaborwanker/Flood-Detection-Using-CNNs/blob/main/VGG_Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#importing all the libraries

import torch
import torchvision
import numpy as np
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from torchvision.utils import make_grid
import cv2
import os
import math
import random
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision.datasets import ImageFolder
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split

In [ ]:
from torchvision import models
from torchvision import transforms
import torch.optim as optim
import time
from PIL import Image

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Images are stored in dataset folder
data_dir =  "/content/drive/MyDrive/NFT DATASET(Only Train)/Train"
dataset = ImageFolder(data_dir,transform = transforms.Compose([transforms.ToTensor(),transforms.Resize((224,224)),transforms.Normalize((0.5, 0.5, 0.5),(0.5, 0.5, 0.5))]))
print(len(dataset))

FileNotFoundError: ignored

In [ ]:
# Sizes of all the sets
train_size = 487
val_size = 62
test_size= 62
batch_size = 50

In [ ]:
#Splitting and Loading the sets
train_data,val_data,test_data = random_split(dataset,[train_size,val_size,test_size])
train_dl = DataLoader(train_data, batch_size, shuffle = True, num_workers = 2, pin_memory = True)
val_dl = DataLoader(val_data, batch_size=16, num_workers = 2, pin_memory = True)
test_dl = DataLoader(test_data, batch_size=16, num_workers = 2, pin_memory = True)

In [ ]:
classes = [0,1]  # Flooded - 0 Non-Flooded-1

In [ ]:
# check GPU availability
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
vgg16 = models.vgg16(pretrained=True)
vgg16.to(device)
print(vgg16)

In [ ]:
# change the number of classes 
vgg16.classifier[6].out_features = 2
# freeze convolution weights
for param in vgg16.features.parameters():
    param.requires_grad = False
# optimizer
optimizer = optim.SGD(vgg16.classifier.parameters(), lr=0.0001, momentum=0.9)
# loss function
criterion = nn.CrossEntropyLoss()

In [ ]:
# training function
def fit(model, train_dataloader):
    model.train()
    train_running_loss = 0.0
    train_running_correct = 0
    for i, data in enumerate(train_dataloader):
        data, target = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        train_running_loss += loss.item()
        _, preds = torch.max(output.data, 1)
        train_running_correct += (preds == target).sum().item()
        loss.backward()
        optimizer.step()
    train_loss = train_running_loss/len(train_dataloader.dataset)
    train_accuracy = 100. * train_running_correct/len(train_dataloader.dataset)
    print(f'Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.2f}')
    
    return train_loss, train_accuracy

In [ ]:
train_loss , train_accuracy = [], []
val_loss , val_accuracy = [], []
start = time.time()
for epoch in range(10):
    print(f'Epoch: {epoch + 1}')
    train_epoch_loss, train_epoch_accuracy = fit(vgg16, train_dl)
    val_epoch_loss, val_epoch_accuracy = validate(vgg16, test_dl)
    train_loss.append(train_epoch_loss)
    train_accuracy.append(train_epoch_accuracy)
    val_loss.append(val_epoch_loss)
    val_accuracy.append(val_epoch_accuracy)
end = time.time()
print((end-start)/60, 'minutes')

In [ ]:
# testing function
def test(model, test_dataloader):
    model.eval()
    val_running_loss = 0.0
    val_running_correct = 0
    for int, data in enumerate(test_dataloader):
        data, target = data[0].to(device), data[1].to(device)
        output = model(data)
        loss = criterion(output, target)
        
        val_running_loss += loss.item()
        _, preds = torch.max(output.data, 1)
        val_running_correct += (preds == target).sum().item()
    
    val_loss = val_running_loss/len(test_dataloader.dataset)
    val_accuracy = 100. * val_running_correct/len(test_dataloader.dataset)
    print(f'Testing Loss: {val_loss:.4f}, Testing Acc: {val_accuracy:.2f}')
    return val_loss, val_accuracy

In [ ]:
# validation function
def validate(model, test_dataloader):
    model.eval()
    val_running_loss = 0.0
    val_running_correct = 0
    for int, data in enumerate(test_dataloader):
        data, target = data[0].to(device), data[1].to(device)
        output = model(data)
        loss = criterion(output, target)
        
        val_running_loss += loss.item()
        _, preds = torch.max(output.data, 1)
        val_running_correct += (preds == target).sum().item()
    
    val_loss = val_running_loss/len(test_dataloader.dataset)
    val_accuracy = 100. * val_running_correct/len(test_dataloader.dataset)
    print(f'Validation Loss: {val_loss:.4f}, Val Acc: {val_accuracy:.2f}')
    return val_loss, val_accuracy

In [ ]:
print(f'Testing Result:')
test_epoch_loss, test_epoch_accuracy = test(vgg16, test_dl)

In [ ]:
plt.figure(figsize=(6, 6))
plt.plot(train_accuracy, color='green', label='train accuracy')
plt.plot(val_accuracy, color='blue', label='validataion accuracy')
plt.legend()
plt.savefig('accuracy.png')
plt.show()

In [ ]:
plt.figure(figsize=(6, 6))
plt.plot(train_loss, color='orange', label='train loss')
plt.plot(val_loss, color='red', label='validataion loss')
plt.legend()
plt.savefig('loss.png')
plt.show()

In [ ]:
dataiter = iter(test_dl)
images, labels = dataiter.next()
classes = ('Flooded', 'Non-Flooded')
# print images
im_grid=torchvision.utils.make_grid(images[0:8])
plt.figure(figsize=(15,15))
plt.imshow(im_grid.permute(1,2, 0))
print('GroundTruth: ', ' '.join('%s\t' % classes[labels[j]] for j in range(8)))
outputs = vgg16(images)
_, predicted = torch.max(outputs, 1)
print('Predicted:   ', ' '.join('%s\t' % classes[predicted[j]] for j in range(8)))